## Import Packages & Dataset

In [43]:
import mlxtend
print(mlxtend.__version__)

0.23.3


In [ ]:
import pandas as pd
from mlxtend.frequent_patterns import apriori, association_rules

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
import networkx as nx

In [18]:
df = pd.read_excel('BACGO 3.0 Dataset for python.xlsx')

## 第一部分：資料清理與格式轉換
目標：將 Excel 資料轉為 "Transaction ID | Items" 格式，適合進行關聯規則分析。

In [19]:
print(df.columns)

Index(['Type', 'Days.for.shipping..real.', 'Days.for.shipment..scheduled.',
       'Benefit.per.order', 'Sales.per.customer', 'Delivery.Status',
       'Late_delivery_risk', 'Category.Id', 'Category.Name', 'Customer.City',
       'Customer.Country', 'Customer.Email', 'Customer.Fname', 'Customer.Id',
       'Customer.Lname', 'Customer.Password', 'Customer.Segment',
       'Customer.State', 'Customer.Street', 'Customer.Zipcode',
       'Department.Id', 'Department.Name', 'Latitude', 'Longitude', 'Market',
       'Order.City', 'Order.Country', 'Order.Customer.Id', 'Order.Id',
       'Order.Item.Cardprod.Id', 'Order.Item.Discount',
       'Order.Item.Discount.Rate', 'Order.Item.Id', 'Order.Item.Product.Price',
       'Order.Item.Profit.Ratio', 'Order.Item.Quantity', 'Sales',
       'Order.Item.Total', 'Order.Profit.Per.Order', 'Order.Region',
       'Order.State', 'Order.Zipcode', 'Product.Card.Id',
       'Product.Category.Id', 'Product.Image', 'Product.Name', 'Product.Price',
       'Pro

In [24]:
df_cleaned = df[['Customer.Id', 'Product.Name']].copy()

In [25]:
df_cleaned.dropna(inplace = True)
df_cleaned.drop_duplicates(inplace = True)

In [27]:
print(df_cleaned.iloc[:5])  # To view the first 5 rows

   Customer.Id                                   Product.Name
0         9083               Perfect Fitness Perfect Rip Deck
1         4741  Under Armour Girls' Toddler Spine Surge Runni
2          639           Nike Men's Dri-FIT Victory Golf Polo
3         9702  Under Armour Girls' Toddler Spine Surge Runni
4         9114           Nike Men's Dri-FIT Victory Golf Polo


In [28]:
# 合併同一交易的產品，生成 "Transaction ID | Items" 格式
df_grouped = df_cleaned.groupby('Customer.Id')['Product.Name'].apply(lambda x: ', '.join(x)).reset_index()
df_grouped.columns = ['Transaction ID', 'Items']

In [29]:
# 儲存為 CSV 以便檢查
df_grouped.to_csv('cleaned_transaction_data.csv', index=False)

In [30]:
print(df_grouped.head())

   Transaction ID                                              Items
0               1                  Nike Men's Free 5.0+ Running Shoe
1               2  Under Armour Girls' Toddler Spine Surge Runni,...
2               3  Nike Men's Free 5.0+ Running Shoe, Nike Men's ...
3               4  Perfect Fitness Perfect Rip Deck, Team Golf Te...
4               5  Perfect Fitness Perfect Rip Deck, Nike Men's C...


## 第二部分：選擇演算法與 Python 實現
推薦演算法：Apriori（適合小到中型數據集）

In [33]:
# 讀取清理後的資料
data = pd.read_csv('cleaned_transaction_data.csv')

In [36]:
# 將資料轉為 One-Hot Encoding 格式
# basket = data['Items'].str.get_dummies(', ')
# 但 Apriori 只接受 True/False 而非 One-Hot Encoding 的 0, 1

# 所以將資料轉為 One-Hot Encoding 格式並轉為布林值
basket = data['Items'].str.get_dummies(', ').astype(bool)

In [41]:
# 使用 Apriori 演算法，找出頻繁項目集
frequent_itemsets = apriori(basket, min_support=0.01, use_colnames=True)
print(frequent_itemsets)

       support                                           itemsets
0     0.022388                          (Bag Boy Beverage Holder)
1     0.024659    (Bridgestone e6 Straight Distance NFL Carolina)
2     0.024822    (Bridgestone e6 Straight Distance NFL San Dieg)
3     0.020604    (Bridgestone e6 Straight Distance NFL Tennesse)
4     0.022794                          (Clicgear 8.0 Shoe Brush)
...        ...                                                ...
6584  0.114536  (Perfect Fitness Perfect Rip Deck, Nike Men's ...
6585  0.110480  (Perfect Fitness Perfect Rip Deck, Nike Men's ...
6586  0.119322  (Perfect Fitness Perfect Rip Deck, Nike Men's ...
6587  0.134247  (Perfect Fitness Perfect Rip Deck, Nike Men's ...
6588  0.102450  (Perfect Fitness Perfect Rip Deck, Nike Men's ...

[6589 rows x 2 columns]


In [ ]:
# 生成關聯規則
# rules = association_rules(frequent_itemsets, metric="lift", min_threshold=1.0)

In [44]:
from mlxtend.frequent_patterns import apriori, association_rules

# 使用 Apriori 演算法，找出頻繁項目集
frequent_itemsets = apriori(basket, min_support=0.01, use_colnames=True)

# 生成關聯規則
rules = association_rules(frequent_itemsets, metric="lift", min_threshold=1.0, num_itemsets=2)

# 查看結果
print(rules)


                                            antecedents  \
0       (Diamondback Women's Serene Classic Comfort Bi)   
1                             (Bag Boy Beverage Holder)   
2           (Field & Stream Sportsman 16 Gun Fire Safe)   
3                             (Bag Boy Beverage Holder)   
4             (Nike Men's CJ Elite 2 TD Football Cleat)   
...                                                 ...   
114679        (Nike Men's CJ Elite 2 TD Football Cleat)   
114680           (Nike Men's Dri-FIT Victory Golf Polo)   
114681      (Field & Stream Sportsman 16 Gun Fire Safe)   
114682               (O'Brien Men's Neoprene Life Vest)   
114683                    (Pelican Sunstream 100 Kayak)   

                                              consequents  antecedent support  \
0                               (Bag Boy Beverage Holder)            0.629786   
1         (Diamondback Women's Serene Classic Comfort Bi)            0.022388   
2                               (Bag Boy Beverag

In [ ]:
frequent_itemsets.to_csv('frequent_itemsets.csv', index=False)

In [45]:
# 過濾規則（置信度 > 50% 且提升度 > 1.2）
filtered_rules = rules[(rules['confidence'] > 0.5) & (rules['lift'] > 1.2)]

# 輸出規則
print(filtered_rules[['antecedents', 'consequents', 'support', 'confidence', 'lift']])

                                              antecedents  \
35        (Bridgestone e6 Straight Distance NFL Carolina)   
163                           (Nike Women's Tempo Shorts)   
369       (Glove It Women's Mod Oval 3-Zip Carry All Gol)   
425                (Hirzl Women's Soffft Flex Golf Glove)   
622          (Team Golf New England Patriots Putter Grip)   
...                                                   ...   
114415  (Diamondback Women's Serene Classic Comfort Bi...   
114416  (Diamondback Women's Serene Classic Comfort Bi...   
114417  (Diamondback Women's Serene Classic Comfort Bi...   
114487  (Under Armour Girls' Toddler Spine Surge Runni...   
114489  (Under Armour Girls' Toddler Spine Surge Runni...   

                                              consequents   support  \
35        (Under Armour Girls' Toddler Spine Surge Runni)  0.016467   
163       (Diamondback Women's Serene Classic Comfort Bi)  0.019630   
369                   (Nike Men's Free 5.0+ Running Sh

## 第三部分：解讀結果與應用

**支持度 (Support)**： 反映商品組合的流行程度（如支持度 0.01 表示該組合在 1% 的交易中出現）。  
**置信度 (Confidence)**： 表示購買 X 後購買 Y 的機率。  
**提升度 (Lift)**： 表示 X 和 Y 的強相關性（提升度 > 1 表示有意義的規則）。  

應用規則時：

- 建議在推薦系統中優先推廣提升度高的商品組合。
- 可設計跨商品類別的促銷活動，例如「買 Footwear，贈 Accessories」。

## 第一步：檢查頻繁項目集結果

在你跑完 Apriori 演算法後，你應該得到頻繁項目集（frequent itemsets）的資料。這些項目集顯示了哪些商品經常一起被購買。

### 1.1 理解頻繁項目集

例如，假設你看到這樣的結果：


itemsets support  
0 (A) 0.05  
1 (B) 0.03  
2 (A, B) 0.02  


這代表：

- 單獨購買 A 的比例是 5%（support = 0.05）。
- 單獨購買 B 的比例是 3%（support = 0.03）。
- 同時購買 A 和 B 的比例是 2%（support = 0.02）。

### 1.2 篩選有意義的項目集

根據你的業務需求，可能不需要考慮所有頻繁項目集。你可以設定支持度閾值（例如，0.01，表示在至少 1% 的交易中出現）來篩選出更有意義的頻繁項目集。

```python
# 設定支持度閾值
min_support = 0.02
filtered_itemsets = frequent_itemsets[frequent_itemsets['support'] >= min_support]

# 顯示篩選後的頻繁項目集
print(filtered_itemsets)

## 第二步：解讀關聯規則

接下來，我們將專注於由 `association_rules` 函數生成的關聯規則。這些規則告訴我們某些產品之間的關聯性。

### 2.1 理解關聯規則

關聯規則表格包含以下幾個關鍵指標：

- **antecedents (前項)**：規則的「如果」部分，表示先購買的商品。
- **consequents (後項)**：規則的「則」部分，表示當購買前項商品後，顧客會購買的商品。
- **support (支持度)**：規則的支持度，表示該規則出現在多少交易中。
- **confidence (置信度)**：表示如果顧客購買了前項商品，購買後項商品的機率。
- **lift (提升度)**：衡量前項和後項之間的相關性，數值越大越顯示兩者有較強的關聯。


| antecedents | consequents | support | confidence | lift |
|--------------|-------------|---------|------------|------|
| (A)         | (B)         | 0.02    | 0.6        | 1.5  |
| (B)         | (A)         | 0.02    | 0.67       | 1.5  |

## 這意味著：

- 當顧客購買了 A 時，有 60% 的機會購買 B，並且這種關聯比隨機購買 A 和 B 更強（提升度為 1.5）。
- 當顧客購買了 B 時，有 67% 的機會購買 A。

### 2.2 篩選規則

為了只關注有商業價值的規則，可以根據 confidence 和 lift 篩選規則。例如：

- 置信度高於 50%。
- 提升度大於 1，顯示有顯著的關聯。

## 第三步：視覺化規則

視覺化可以幫助更直觀地理解哪些商品經常一起購買，以及它們之間的關聯性。

### 3.1 使用 Matplotlib 繪製條形圖

可以繪製 confidence 或 lift 的條形圖，顯示規則的強度。

In [ ]:

# 繪製規則的置信度
plt.figure(figsize=(10,6))
plt.bar(filtered_rules['antecedents'].astype(str) + " -> " + filtered_rules['consequents'].astype(str),
        filtered_rules['confidence'], color='lightblue')
plt.xticks(rotation=90)
plt.xlabel('Rules')
plt.ylabel('Confidence')
plt.title('Association Rules Confidence')
plt.show()


### 3.2 使用 networkx 繪製關聯網絡  

如果有多條關聯規則，可以使用網絡圖來顯示不同商品之間的聯繫。

In [ ]:

# 建立網絡圖
G = nx.Graph()

# 加入節點和邊（商品）
for _, rule in filtered_rules.iterrows():
    G.add_edge(str(rule['antecedents']), str(rule['consequents']), weight=rule['lift'])

# 繪製圖
plt.figure(figsize=(12,12))
pos = nx.spring_layout(G, seed=42)
nx.draw_networkx_nodes(G, pos, node_size=3000, node_color='lightgreen', alpha=0.7)
nx.draw_networkx_edges(G, pos, width=2.0, alpha=0.7)
nx.draw_networkx_labels(G, pos, font_size=12, font_color='black')
plt.title("Association Rules Network")
plt.show()


## 第四步：應用與商業決策

### 推薦系統
根據頻繁出現的商品組合設計推薦策略，例如，當顧客購買了 X 商品時，推薦 Y 商品。

### 促銷活動
可以針對提升度高的規則設計促銷活動，推動商品捆綁銷售或聯合促銷。